In [1]:
from helper_functions import * 
from datetime import datetime

In [4]:
def general_extractor(metadata, field):
    data = []
    for reference in campaign_metadata:
        value = reference['metadata'].get(field,'')
        if value:
            data.append(value)
    return data 

# User Input

In [2]:
short_name='above'
dep_start = datetime(2015,1,2)
dep_end = datetime(2016,1,1)

# Pipeline

In [3]:
# initial api query

campaign_tree = ingest_campaign(short_name)
campaign_metadata = campaign_xlm_json(campaign_tree)

In [20]:
# for use in deployment/flight level assignments
filtered_metadata = date_filter(campaign_metadata, dep_start, dep_end)

In [6]:
db = {}

In [9]:
# ContactPersons
# possible contact persons, might be suggested for serveral fields
# Campaign.technical_contact
# Campaign.project_lead
# Campaign.program_lead
# Instrument.technical_contact
# Instrument.lead_investigator
# flight.platform_technical_contact

db['campaign']={}
db['campaign']['contacts']=general_extractor(campaign_metadata, 'ContactPersons')

In [10]:
# Abstract
# could help when writing the description 
db['campaign']['description_long']=general_extractor(campaign_metadata, 'Abstract')

In [15]:
# DOI
# this doesn't have a direct match. Need to ask Stephanie if she wants it
dois = general_extractor(campaign_metadata, 'DOI')
processed_dois=[item.get('DOI',None) for item in dois]
db['campaign']['dois'] = processed_dois

In [16]:
# SpatialExtents
db['campaign']['spatial_bounds']=general_extractor(campaign_metadata, 'SpatialExtents')
# TODO: this needs more processing in the combine_spatial_extents() function 

In [18]:
# LocationKeywords
# this doesn't have a direct match. Need to ask Stephanie if she wants it
# otherwise, we can parse these and put them into the campaign.region_description
db['campaign']['gcmd_region']=general_extractor(campaign_metadata, 'LocationKeywords')
# TODO: might need to be processed by extracting the 'Subregion2' 

In [23]:
# Platforms
db['deployment'] = extract_inst_plat(filtered_metadata)
# TODO: maybe make a flight from each platform and assign it instruments?

In [38]:
# CollectionCitations
citations = general_extractor(campaign_metadata, 'CollectionCitations')
db['campaign']['publications']=[
    citation 
        for citation_list in citations
            for citation in citation_list 
                if citation.get('Title')
]
# this has more information in it than we currently track for campaign.publications...
# maybe we can add some more granularity to that field?

In [39]:
# ScienceKeywords
db['campaign']['gcmd_phenomena']=general_extractor(campaign_metadata, 'ScienceKeywords')

In [40]:
# RelatedUrls
db['campaign']['other_resources']=general_extractor(campaign_metadata, 'RelatedUrls')
# this contains a 'Description' of the resource as well as the 'URL'
# perhaps we should implement an other_resource description in our db?

In [63]:
# DataCenters
# There are multiple DAAC roles, ARCHIVER, DISTRIBUTOR, PROCESSOR, ORIGINATOR.
# Which ones do we care about

role_filter=['ARCHIVER', 'DISTRIBUTOR', 'PROCESSOR', 'ORIGINATOR']

mega_daac_list=general_extractor(campaign_metadata, 'DataCenters')
db['campaign']['repositories']=[
    daac['ShortName']
        for daac_list in mega_daac_list
             for daac in daac_list
                 if daac['Roles'][0] in role_filter
        ]